In [87]:
##### Part 0: housekeeping 
import csv
import pandas as pd
import copy
import numpy as np
from scipy import stats
%pprint

Pretty printing has been turned ON


In [85]:
def ttest_is_significant (series1, series2, threshold=0.05, equal_var=False):
    #statistic = stats.ttest_ind(series1, series2, equal_var=equal_var).statistic
    #pvalue = stats.ttest_ind(series1, series2, equal_var=equal_var).pvalue
    ret = stats.ttest_ind(series1, series2, equal_var=equal_var)
    #print (ret.pvalue)
    if ret.pvalue > threshold: 
        result = 1  # pass, i.e, 'Insignificant'
    else:
        result = 0  # fail, i.e, 'Significantly different distribution'
    #print (result)
    return result

In [2]:
#set the mode to view the df
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
def rearrange_col(dataframe, col_to_move, reference_col, right=True):
    col_list = dataframe.columns.values.tolist()
    col_list2 = [x for x in col_list if x != col_to_move]
    reference_idx = col_list2.index(reference_col)
    if right==True:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.append(col_to_move)
        return dataframe[col_list3]
    else:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.insert(-2, col_to_move)
        return  dataframe[col_list3]

In [4]:
##### Part1: load spreadsheet, list 

In [5]:
raw0 = pd.read_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Verblists/verblist_v2.16_wb.xlsx')
#raw0['in'] = ''
#raw = rearrange_col(raw0, 'in', 'Index')
#raw = copy.deepcopy(raw)

In [6]:
###load candidate list
list1 = '/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Verblists/63_80_70_list1.txt'

##load a dict from list
import ast
file = open(list1, "r")
contents = file.read()
dic = ast.literal_eval(contents)

In [7]:
verbs_in = dic['unergative'] + dic['unaccusative'] + dic['transitive']

### update raw0['In']; 1 if Lemma in verbs_in, otherwise 0
raw0['in'] = np.where(raw0['Lemma'].isin(verbs_in), 1, 0)

#raw['In'] = [1 if x in verbs_in else 0 for x in raw['Lemma']]
#raw['In'] = np.where(raw['Lemma'].isin(verbs_in), 1, 0)
#raw[raw['Lemma'].isin(verbs_in)] 
# raw[raw['Lemma'].isin(verbs_in)].loc[:, 'In'] = 1
# raw[~raw['Lemma'].isin(verbs_in)].loc[:, 'In'] = 0

In [ ]:
raw0.head()

In [8]:
###further cleaning 
raw = raw0.sort_values(by=['in', 'SynClassNew', 'GrandIndex'], ascending=[False, True, True])
raw.drop(columns=['Index'], inplace=True)
# raw = raw.reset_index(drop=True)
#raw = rearrange_col(raw0, 'in', 'Index')
# raw = raw.reset_index(drop=True)

raw_in = raw[raw['in']==1]
raw_out = raw[raw['in']==0]

raw_in_unerg = raw_in[raw_in['SynClassNew']=='unergative'] #####choose only those selected
raw_in_unacc = raw_in[raw_in['SynClassNew']=='unaccusative']
raw_in_trans = raw_in[raw_in['SynClassNew']=='transitive']

# raw_in = raw[raw['in']==1]

In [9]:
raw_in_trans.head()

,Lemma,GrandIndex,LogFreqHAL,Concreteness,AgeofAcqsn,Database,Trans_Intrans,SynClassLitrt,SynClassMemo,SynClassSK,SynClassNew,unerg_pb,unacc_pb,tr_pb,totalN_pb,most_frequent_pb,intr_g,tr_g,totalN_g,most_frequent_g,Note,SynClassCorpus,SynClassFinal,intr_google,trans_google,google1950_intrans,google1950_trans,AspClass_Consensus,SemanticType_JB,SemanticType_JB_edit,Typicality_JB,SemanticType_LC,Typicality_LC,SemClass_Consensus,CONSENSUS_JB,SemanticTypeLevin,SemanticTypeEntry,Percentage_dom_PoS,Length,Ortho_N,Phono_N,OLD,PLD,NPhon,NSyll,I_Mean_RT,I_Mean_Accuracy,I_NMG_Mean_RT,I_NMG_Mean_Accuracy,N2_F,N3_F,in
3,play,4,11.915,3.24,4.10,CREA,both,unergative,"unergative, transitive",unergative,transitive,9.7,15.0,72.0,186.0,tr,33.0,57.0,150.0,tr,NaN,tr,unergative,55.61,44,47.66,52.34,Process,social event,social event,5.0,social or interpersonal event,3.0,social or interpersonal event,social or interpersonal event,"verbs of creating and transformation, verbs of social interaction","26.7perform, 36.3meet",0.81,4,6,15,1.45,1.10,3,1,537.909,1.0,566.077,0.963,781.22,329.47,1
89,open,90,11.885,3.21,5.00,CREA,both,unaccusative,unaccusative,unaccusative,transitive,1.7,51.0,43.0,242.0,unacc,NaN,NaN,NaN,NaN,NaN,unacc,unaccusativefailed,33.61,66.19,36.59,63.41,Achievement,"ch of appearance, obj dir action or transfer","ch of appearance, obj dir action or transfer",3.0,obj dir action,4.0,obj-directed action,obj-directed action,"verbs of involving the body, verbs of change of state, verbs of existence, appearance, disappearance, and occurrence","40.3.2crane, 45.4state, 47.6balance, 48.1.1appear",0.68,4,4,4,1.75,1.75,4,2,554.471,1.0,525.929,1.000,3269.84,304.48,1
98,resume,99,11.295,4,13.47,VerbNet,both,NaN,unaccusative,unaccusative,transitive,NaN,34.0,66.0,71.0,tr,NaN,NaN,NaN,NaN,polysemous,tr,unaccusativefailed,10.81,89.13,29.86,70.14,Achievement,aspectual,aspectual,5.0,aspectual,5.0,aspectual,aspectual,aspectual verbs,55.1begin,0.65,6,0,0,1.90,2.20,5,2,588.364,1.0,649.040,0.926,937.52,146.38,1
114,click,115,9.906,4.03,6.94,VerbNet,both,NaN,unaccusative? transitive?,unaccusative,transitive,NaN,NaN,100.0,1.0,tr,NaN,NaN,NaN,NaN,NaN,tr,unaccusativefailed,24.01,75.99,25.85,74.15,Semelfactive,emission,emission,4.0,obj dir action/emission,3.0,obj-directed action,obj-directed action,"verbs of involving the body, verbs of emission","40.3.2crane, 43.2bang",0.60,5,8,16,1.55,1.10,4,1,670.257,1.0,614.966,1.000,1670.75,126.03,1
131,flip,132,8.699,3.97,5.11,VerbNet,both,NaN,unaccusative,unaccusative,transitive,NaN,20.0,80.0,5.0,tr,NaN,NaN,NaN,NaN,NaN,tr,unaccusative,58.27,41.25,47.90,52.10,Achievement,obj dir action or transfer,obj dir action or transfer,5.0,manner of motion/obj dir action,3.0,obj-directed action,obj-directed action,verbs of throwing,17.1throw,0.78,4,6,10,1.60,1.40,4,1,597.939,1.0,557.481,0.964,166.54,19.10,1


In [10]:
raw_in_unerg.reset_index(drop=True, inplace=True)
raw_in_unacc.reset_index(drop=True, inplace=True)
raw_in_trans.reset_index(drop=True, inplace=True) ##drop=True so that original index is really dropped

In [11]:
#now add the "cleaned" index to an ordinary column
raw_in_unerg.reset_index(inplace=True) 
raw_in_unacc.reset_index(inplace=True)
raw_in_trans.reset_index(inplace=True)

##do the same for raw_out
raw_out.reset_index(inplace=True)

In [12]:
raw_out.head()

,index,Lemma,GrandIndex,LogFreqHAL,Concreteness,AgeofAcqsn,Database,Trans_Intrans,SynClassLitrt,SynClassMemo,SynClassSK,SynClassNew,unerg_pb,unacc_pb,tr_pb,totalN_pb,most_frequent_pb,intr_g,tr_g,totalN_g,most_frequent_g,Note,SynClassCorpus,SynClassFinal,intr_google,trans_google,google1950_intrans,google1950_trans,AspClass_Consensus,SemanticType_JB,SemanticType_JB_edit,Typicality_JB,SemanticType_LC,Typicality_LC,SemClass_Consensus,CONSENSUS_JB,SemanticTypeLevin,SemanticTypeEntry,Percentage_dom_PoS,Length,Ortho_N,Phono_N,OLD,PLD,NPhon,NSyll,I_Mean_RT,I_Mean_Accuracy,I_NMG_Mean_RT,I_NMG_Mean_Accuracy,N2_F,N3_F,in
0,266,use,267,13.426,2.78,4.50,CREA,transitive only,transitive,transitive,transitive,?transitive,0.12,34.00,60.0,839.0,tr,NaN,NaN,NaN,NaN,NaN,tr,transitive,28.05,71.84,60.74,39.26,Process,obj dir action or transfer,obj dir action or transfer,5.0,obj dir action,3.0,obj-directed action,obj-directed action,"verbs with predicative complements, measure verbs","29.2characterize, 54.3fit",0.86,3,1,33,1.65,1.00,3,1,576.710,0.94,618.346,1.0,467.49,467.49,0
1,268,want,269,13.138,1.93,4.16,CREA,both,NaN,transitive,transitive,?transitive,0.16,0.81,98.0,617.0,tr,1.6,29.0,126.0,tr,psych,tr,NaN,86.73,13.24,88.52,11.48,State,emotion,emotion,4.0,emotion,5.0,emotion,emotion,"verbs with predicative complements, verbs of desire","29.1appoint, 32.1want",1.00,4,16,14,1.30,1.55,4,1,596.212,1.00,580.240,1.0,2200.56,607.27,0
2,269,find,270,12.845,2.63,5.78,CREA,transitive only,transitive,transitive,transitive,?transitive,2.00,8.50,78.0,457.0,tr,1.5,87.0,200.0,tr,NaN,tr,transitive,39.38,60.15,51.56,48.44,Achievement,cognitive event,cognitive event,3.0,cognitive event,4.0,cognitive event,cognitive event,"verbs of change of possession, verbs with predicative complements","13.5.1get, 29.4declare",1.00,4,15,26,1.15,1.15,4,1,619.562,0.94,580.333,1.0,2150.47,997.75,0
3,270,help,271,12.723,2.56,3.65,CREA,both,NaN,transitive (or covert obj),transitive,?transitive,4.20,5.30,88.0,526.0,tr,1.5,15.0,200.0,tr,NaN,tr,NaN,59.84,40.04,72.41,27.59,Process,social event,social event,5.0,social or interpersonal event,5.0,social or interpersonal event,social or interpersonal event,not found,NaN,0.80,4,7,9,1.55,1.50,4,1,601.970,0.94,576.074,1.0,1899.11,536.54,0
4,275,write,276,11.795,4.22,4.89,CREA,both,NaN,transitive (or covert obj),transitive,?transitive,13.00,18.00,56.0,208.0,tr,30.0,41.0,131.0,tr,NaN,tr,NaN,58.74,40.1,69.75,30.25,Process,creation,creation,4.0,creation,4.0,creation or destruction,creation or destruction,"image creating verbs, verbs of creating and transformation, verbs of communication","25.2scribble, 26.7perform, 37.1quote",1.00,5,5,40,1.55,1.00,3,1,608.000,1.00,597.926,1.0,1286.52,437.95,0


In [13]:
raw_in_unerg.shape[1] == raw_out.shape[1]

True

In [29]:
### join datasets
raw_clean = pd.concat([raw_in_unerg, raw_in_unacc, raw_in_trans, raw_out])
#raw_clean.reset_index(drop=True, inplace=True)
raw_clean = rearrange_col(raw_clean, 'index', 'GrandIndex')
raw_clean = rearrange_col(raw_clean, 'in', 'index')
raw_clean = raw_clean.set_index('Lemma')

In [30]:
raw_clean

,GrandIndex,index,in,LogFreqHAL,Concreteness,AgeofAcqsn,Database,Trans_Intrans,SynClassLitrt,SynClassMemo,SynClassSK,SynClassNew,unerg_pb,unacc_pb,tr_pb,totalN_pb,most_frequent_pb,intr_g,tr_g,totalN_g,most_frequent_g,Note,SynClassCorpus,SynClassFinal,intr_google,trans_google,google1950_intrans,google1950_trans,AspClass_Consensus,SemanticType_JB,SemanticType_JB_edit,Typicality_JB,SemanticType_LC,Typicality_LC,SemClass_Consensus,CONSENSUS_JB,SemanticTypeLevin,SemanticTypeEntry,Percentage_dom_PoS,Length,Ortho_N,Phono_N,OLD,PLD,NPhon,NSyll,I_Mean_RT,I_Mean_Accuracy,I_NMG_Mean_RT,I_NMG_Mean_Accuracy,N2_F,N3_F
Lemma,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
work,1,0,1,13.020,3.48,5.86,CREA,both,unergative,unergative,unergative,unergative,78.0,4.40,16.0,431.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,88.78,11.17,90.01,9.99,Process,other - general process,other - general process,5.0,other,NaN,other,other,verbs of creating and transformation,26.5knead,0.57,4,9,25,1.50,1.20,3,1,633.152,1.00,521.889,1.000,2421.10,1034.67
run,3,1,1,12.316,4.31,4.47,CREA,both,unergative,unergative,unergative,unergative,18.0,34.00,44.0,374.0,tr,NaN,NaN,NaN,NaN,NaN,tr,unergative,76.27,23.65,71.00,29.00,Process,body mvt or posture,body movement or posture,5.0,manner of motion,5.0,manner of motion,manner of motion,"verbs of creating and transformation, verbs of existence, verbs of existence, verbs of motion","26.3prepare, 47.5.1swarm, 47.7meander, 51.3.2run",0.89,3,19,40,1.00,1.00,3,1,596.647,1.00,559.955,1.000,386.53,229.70
live,5,2,1,11.763,3.57,6.10,CREA,intransitive only,unergative/unaccusative,unergative PP arg,unergative,unergative,95.0,0.78,4.7,129.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,94.18,5.81,92.30,7.70,State,existence,existence,5.0,existence,5.0,existence,existence,"verbs of ingesting, lodge verbs, verbs of existence","39.6gorge, 46lodge, 47.1exist",0.89,4,16,21,1.15,1.25,3,1,629.886,1.00,553.769,0.963,3645.64,606.44
leave,6,3,1,11.245,2.53,5.58,CREA,both,NaN,"unergative PP arg, transitive",unergative,unergative,13.0,8.80,40.0,354.0,tr,10.0,49.0,200.0,tr,polysemous,tr,unergativefailed,38.02,61.13,57.57,42.43,Achievement,ch of location,ch of location,5.0,ch of location,5.0,ch of location,ch of location,"verbs of change of possession, verbs of change of possession, verbs of change of possession, hold and keep verbs, verbs of motion, verbs of motion","13.3FUT:have, 13.4.1entrust, 13.5.1get, 15.2keep, 51.1advance, 51.2abandon",0.97,5,4,33,1.70,1.00,3,1,565.818,0.97,617.846,1.000,1630.15,502.36
speak,7,4,1,10.891,3.7,3.56,CREA,both,unergative,"unergative, transitive",unergative,unergative,87.0,NaN,13.0,69.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,87.52,12.45,87.10,12.90,Process,communication,communication,5.0,communication,5.0,communication,communication,verbs of communication,37.5talk,1.00,5,4,12,1.70,1.30,4,1,598.719,1.00,645.130,1.000,973.45,284.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bathe,65,64,0,6.507,4.19,5.00,VerbNet,both,NaN,"unergative, transitive",unergative,unergative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,unergative,80.05,19.95,77.22,22.78,Process,body mvt or posture,body movement or posture,3.0,other,NaN,other,other,"verbs of putting, verbs of grooming and bodily care","09.8fill, 41.1.1dress",1.00,5,3,15,1.75,1.25,3,1,800.296,0.84,684.593,0.964,518.39,19.06
squeak,72,71,0,6.246,4.07,5.84,VerbNet,intransitive only,NaN,unergative,unergative,unergative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,unergative,97.43,2.57,94.52,5.48,Semelfactive,emission,emission,5.0,oral sound production/emission,4.0,emission,emission,"verbs of communication, verbs of sounds made by animals, verbs of emission","37.3snap/cackle, 38bark, 43.2bang",0.56,6,1,3,2.25,1.75,5,1,715.500,0.97,754.920,1.000,157.42,34.22
flinch,77,76,0,5.875,3.59,7.89,VerbNet,intransitive onl

In [31]:
#### save the updated spreadsheet to csv
raw_clean.to_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Verblists/candidate1.xlsx')

In [101]:
####################### PART 2: this is the place to make small adjustments ###################
raw_adjust = pd.read_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Verblists/candidate1.xlsx')
raw_adjust = raw_adjust.set_index('Lemma')

In [123]:
item_to_exclude = ['lie', 'leave', 'stand', 'tear']
for i in item_to_exclude:
    raw_adjust.at[i, 'in'] = 0
# raw_adjust.at['lie', 'in'] = 0
# raw_adjust.at['leave', 'in'] = 0
# raw_adjust.at['leave', 'in'] = 0

In [124]:
#raw_adjust.loc['stand']['in']

0

In [125]:
col_to_test = ['LogFreqHAL', 'AgeofAcqsn', 'Length', 'Ortho_N', 'Phono_N', 'OLD', 'PLD', 'NPhon', 'NSyll', \
               'I_Mean_RT', 'I_Mean_Accuracy', 'I_NMG_Mean_RT', 'I_NMG_Mean_Accuracy', 'N2_F', 'N3_F'] 
##Concreteness, Percentage_dom_PoS are not includeed

In [126]:
len(col_to_test)

15

In [129]:
######let's see if df with adjustment passes t-tests
from scipy import stats
raw_in_update = raw_adjust[raw_adjust['in']==1]
unerg_df = raw_in_update[raw_in_update['SynClassNew']=='unergative']
unacc_df = raw_in_update[raw_in_update['SynClassNew']=='unaccusative']
trans_df = raw_in_update[raw_in_update['SynClassNew']=='transitive']

print ('(1: pvalue>0.05, 0: pvalue =<0.05) \n')
for c in col_to_test: 
    unerg = unerg_df[c].tolist()
    unacc = unacc_df[c].tolist()
    trans = trans_df[c].tolist()

    print (c, 'unerg-unacc', ttest_is_significant(unerg, unacc))
    print (c, 'unacc-trans', ttest_is_significant(unacc, trans))
    print (c, 'unerg-trans', ttest_is_significant(unerg, trans))
    print ('========================================================')

print ('unergN:', unerg_df.shape[0])
print ('unaccN:', unacc_df.shape[0])
print ('transN:', trans_df.shape[0])
print ('totalN:',  unerg_df.shape[0]+unacc_df.shape[0]+trans_df.shape[0])

(1: pvalue>0.05, 0: pvalue =<0.05) 

LogFreqHAL unerg-unacc 1
LogFreqHAL unacc-trans 0
LogFreqHAL unerg-trans 0
AgeofAcqsn unerg-unacc 1
AgeofAcqsn unacc-trans 1
AgeofAcqsn unerg-trans 1
Length unerg-unacc 1
Length unacc-trans 1
Length unerg-trans 1
Ortho_N unerg-unacc 1
Ortho_N unacc-trans 1
Ortho_N unerg-trans 1
Phono_N unerg-unacc 1
Phono_N unacc-trans 1
Phono_N unerg-trans 1
OLD unerg-unacc 1
OLD unacc-trans 1
OLD unerg-trans 1
PLD unerg-unacc 1
PLD unacc-trans 1
PLD unerg-trans 1
NPhon unerg-unacc 1
NPhon unacc-trans 1
NPhon unerg-trans 1
NSyll unerg-unacc 1
NSyll unacc-trans 1
NSyll unerg-trans 1
I_Mean_RT unerg-unacc 1
I_Mean_RT unacc-trans 1
I_Mean_RT unerg-trans 1
I_Mean_Accuracy unerg-unacc 1
I_Mean_Accuracy unacc-trans 1
I_Mean_Accuracy unerg-trans 1
I_NMG_Mean_RT unerg-unacc 1
I_NMG_Mean_RT unacc-trans 1
I_NMG_Mean_RT unerg-trans 1
I_NMG_Mean_Accuracy unerg-unacc 1
I_NMG_Mean_Accuracy unacc-trans 1
I_NMG_Mean_Accuracy unerg-trans 1
N2_F unerg-unacc 1
N2_F unacc-trans 1
N2_F

In [75]:
#'lie' in unerg_df.index.tolist()

In [ ]:
###################### PART 3: describe the dataset ###################################
table_raw = pd.crosstab(raw_in.SemClass_Consensus, raw_in.SynClassNew, margins=True) #looks super messy due to messy labels

In [ ]:
unerg